# load dependencies


In [1]:
import _resume_eval_import_helper

In [2]:
import os 
import tiktoken 
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv, find_dotenv
from uuid import uuid4
import pandas as pd
import json
from langchain_core.prompts import PromptTemplate
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI 
from langchain_anthropic import ChatAnthropic
from langchain_ollama import ChatOllama

from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from prompts.resume_eval import RESUME_EVALUATION_PROMPT
from langchain_core.runnables.base import RunnableSequence

import logging
import time
from datetime import datetime 
from typing import Dict, Any, Union, List, Tuple

from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm.auto import tqdm

In [3]:
load_dotenv(find_dotenv("../../.env"))

True

# helper functions


In [4]:
# generate unique id
def generate_unique_id(_):
    return str(uuid4())

# compare model outputs
def compare_model_outputs(model_results):
    """
    Compare original and recalibrated scores from different models and output a pretty table.
    
    :param model_results: Dict with model names as keys and JSON outputs as values
    :return: pandas DataFrame with a pretty table comparison
    """
    comparison_data = []
    
    score_types = [
        "technical_skills",
        "soft_skills",
        "required_experience",
        "qualifications"
    ]
    
    for model, result in model_results.items():
        # Ensure result is a dictionary
        if isinstance(result, str):
            result = json.loads(result)
        
        model_data = {"Model": model}
        
        # Extract original scores
        original_scores = result.get("resume_evaluation", {}).get("original_scores", {})
        
        # Extract recalibrated scores
        recalibrated_scores = result.get("recalibrated_scores", {})
        
        for score_type in score_types:
            model_data[f"original_{score_type}"] = original_scores.get(score_type, "N/A")
            model_data[f"recalibrated_{score_type}"] = recalibrated_scores.get(score_type, "N/A")
        
        # Add suitability
        model_data["suitability"] = result.get("assessment", {}).get("suitability", "N/A")
        
        comparison_data.append(model_data)
    
    # Create DataFrame
    df = pd.DataFrame(comparison_data)
    
    # Reorder columns
    column_order = ["Model"] + [f"{prefix}_{score_type}" for score_type in score_types for prefix in ["original", "recalibrated"]] + ["suitability"]
    df = df[column_order]
    
    return df

# evaluate resume 
def evaluate_resume(model_tuples: List[Tuple[str, RunnableSequence]], job_description: str, resume: str, job_id: str, cv_id: str, output_dir: str) -> Union[pd.DataFrame, None]:
    model_results = {}
    for model_name, grader in model_tuples:
    # for model_name, grader in [("llama3", llama3_grader), ("gpt", gpt_grader), ("anthropic", anthropic_grader)]:
        try:
            result = grader.invoke({"job_description": job_description, "resume": resume})
            model_results[model_name] = result

            # save model result 
            json_file = os.path.join(output_dir, f"{job_id}_{cv_id}_{model_name}.json")
            with open(json_file, "w") as f:
                json.dump(result, f, indent=4)
            
            time.sleep(2.1)  # Add a small delay to avoid rate limiting
        except Exception as e:
            error_msg = f"Error with {model_name} for job_id: {job_id}, cv_id: {cv_id}. Error: {str(e)}"
            logging.error(error_msg)
            print(error_msg)

    if not model_results:
        error_msg = f"All models failed for job_id: {job_id}, cv_id: {cv_id}."
        logging.error(error_msg)
        print(error_msg)
        return None
    
    # try:
        
    #     comparison_df = compare_model_outputs(model_results)
        
    #     if not os.path.exists(output_dir):
    #         os.makedirs(output_dir)
        
    #     # save comparison result 
    #     csv_file = os.path.join(output_dir, f"{job_id}_{cv_id}.csv")
    #     comparison_df.to_csv(csv_file, index=False)
        
    #     logging.info(f"Successfully processed and saved results for job_id: {job_id}, cv_id: {cv_id}.")
        
    #     # return comparison_df
        
    # except Exception as e:
    #     error_msg = f"Error in processing or saving results for job_id: {job_id}, cv_id: {cv_id}. Error: {error_msg}."
    #     logging.error(error_msg)
    #     print(error_msg)
    #     return None

def generate_unique_id(_):
    return str(uuid4())

# compare model outputs
def compare_model_outputs(model_results):
    """
    Compare original and recalibrated scores from different models and output a pretty table.
    
    :param model_results: Dict with model names as keys and JSON outputs as values
    :return: pandas DataFrame with a pretty table comparison
    """
    comparison_data = []
    
    score_types = [
        "technical_skills",
        "soft_skills",
        "required_experience",
        "qualifications"
    ]
    
    for model, result in model_results.items():
        # Ensure result is a dictionary
        if isinstance(result, str):
            result = json.loads(result)
        
        model_data = {"Model": model}
        
        # Extract original scores
        original_scores = result.get("resume_evaluation", {}).get("original_scores", {})
        
        # Extract recalibrated scores
        recalibrated_scores = result.get("recalibrated_scores", {})
        
        for score_type in score_types:
            model_data[f"original_{score_type}"] = original_scores.get(score_type, "N/A")
            model_data[f"recalibrated_{score_type}"] = recalibrated_scores.get(score_type, "N/A")
        
        # Add suitability
        model_data["suitability"] = result.get("assessment", {}).get("suitability", "N/A")
        
        comparison_data.append(model_data)
    
    # Create DataFrame
    df = pd.DataFrame(comparison_data)
    
    # Reorder columns
    column_order = ["Model"] + [f"{prefix}_{score_type}" for score_type in score_types for prefix in ["original", "recalibrated"]] + ["suitability"]
    df = df[column_order]
    
    return df

# evaluate resume 
def evaluate_resume(model_tuples: List[Tuple[str, RunnableSequence]], job_description: str, resume: str, job_id: str, cv_id: str, output_dir: str) -> Union[pd.DataFrame, None]:
    model_results = {}
    for model_name, grader in model_tuples:
    # for model_name, grader in [("llama3", llama3_grader), ("gpt", gpt_grader), ("anthropic", anthropic_grader)]:
        try:
            result = grader.invoke({"job_description": job_description, "resume": resume})
            model_results[model_name] = result

            # save model result 
            json_file = os.path.join(output_dir, f"{job_id}_{cv_id}_{model_name}.json")
            with open(json_file, "w") as f:
                json.dump(result, f, indent=4)
            
            time.sleep(2.1)  # Add a small delay to avoid rate limiting
        except Exception as e:
            error_msg = f"Error with {model_name} for job_id: {job_id}, cv_id: {cv_id}. Error: {str(e)}"
            logging.error(error_msg)
            print(error_msg)

    if not model_results:
        error_msg = f"All models failed for job_id: {job_id}, cv_id: {cv_id}."
        logging.error(error_msg)
        print(error_msg)
        return None
    
    # try:
        
    #     comparison_df = compare_model_outputs(model_results)
        
    #     if not os.path.exists(output_dir):
    #         os.makedirs(output_dir)
        
    #     # save comparison result 
    #     csv_file = os.path.join(output_dir, f"{job_id}_{cv_id}.csv")
    #     comparison_df.to_csv(csv_file, index=False)
        
    #     logging.info(f"Successfully processed and saved results for job_id: {job_id}, cv_id: {cv_id}.")
        
    #     # return comparison_df
        
    # except Exception as e:
    #     error_msg = f"Error in processing or saving results for job_id: {job_id}, cv_id: {cv_id}. Error: {error_msg}."
    #     logging.error(error_msg)
    #     print(error_msg)
    #     return None




# global variables


In [5]:
# get current time 
current_time = datetime.now().strftime(("%Y%m%d_%H%M"))
output_dir = f"./output_{current_time}/"

# create output directory 
os.makedirs(output_dir, exist_ok=True)

temperature = 0
max_tokens = 2048

# set up logger 
log_file = os.path.join(output_dir, "evaluation_log.txt")
logging.basicConfig(format="%(asctime)s - %(levelname)s - %(message)s", level=logging.INFO, filename=log_file, datefmt="%Y-%m-%d %H:%M:%S")

# import dataset


In [41]:
data_path = "../repos/Resume-Screening-RAG-Pipeline/data/supplementary-data/"
synthetic_data_path = "../repos/Resume-Screening-RAG-Pipeline/data/main-data/"

job_description_path = data_path + "job_title_des.csv"
cleaned_resume_path = data_path + "cleaned_resume.csv"
synthetic_resume_path = synthetic_data_path + "synthetic-resumes.csv"

cleaned_resume = pd.read_csv(cleaned_resume_path)
synthetic_resumes = pd.read_csv(synthetic_resume_path)
job_description = pd.read_csv(job_description_path)

In [42]:
len(cleaned_resume), len(synthetic_resumes), len(job_description)

(162, 1000, 2277)

In [43]:
cleaned_resume["Category"].value_counts()

Category
Java Developer               13
Database                     11
Data Science                 10
Advocate                     10
DotNet Developer              7
Hadoop                        7
DevOps Engineer               7
Automation Testing            7
Testing                       7
Python Developer              6
Arts                          6
Health and fitness            6
Civil Engineer                6
HR                            6
SAP Developer                 6
Business Analyst              6
Electrical Engineering        5
Network Security Engineer     5
Mechanical Engineer           5
ETL Developer                 5
Blockchain                    5
Sales                         5
Operations Manager            4
Web Designing                 4
PMO                           3
Name: count, dtype: int64

In [9]:
cleaned_resume.columns, synthetic_resumes.columns, job_description.columns

(Index(['Category', 'Resume', 'ID'], dtype='object'),
 Index(['ID', 'Resume'], dtype='object'),
 Index(['Job Title', 'Job Description'], dtype='object'))

In [10]:
# apply unique id 
job_description["Job ID"] = job_description["Job Title"].apply(generate_unique_id)

## data preparation


In [11]:
# categorize the resumes 
# we can sample the job pool to reduce the cost 
tech_jobs = ['Data Science', 
      'Java Developer', 'Business Analyst',
      'SAP Developer','Python Developer', 'DevOps Engineer',
      'Network Security Engineer','Database', 'Hadoop',
      'ETL Developer', 'DotNet Developer', 'Blockchain']

non_tech_jobs = [
  'HR', 'Advocate', 'Arts', 'Web Designing', 'Mechanical Engineer', 'Sales', 'Health and fitness',
  'Civil Engineer',  'Automation Testing', 'Electrical Engineering',
  'Operations Manager',  'PMO',
]

tech_pool = cleaned_resume[cleaned_resume["Category"].isin(tech_jobs)] # 88
non_tech_pool = cleaned_resume[cleaned_resume["Category"].isin(non_tech_jobs)] # 67

In [12]:
tech_pool = cleaned_resume[cleaned_resume["Category"].isin(tech_jobs)] # 88
non_tech_pool = cleaned_resume[cleaned_resume["Category"].isin(non_tech_jobs)] # 67

reduced_tech_pool = tech_pool.groupby("Category").apply(lambda x: x.sample(frac=0.5, random_state=42)).reset_index(drop=True)
reduced_non_tech_pool = non_tech_pool[non_tech_pool["Category"].isin(["HR", "Advocate", "Arts"])].sample(3, random_state=42)

talent_pool = pd.concat([reduced_tech_pool, reduced_non_tech_pool]).sample(frac=1, random_state=42).reset_index(drop=True)

/var/folders/x3/z5c7sxc131d8t8hhqfw7x3780000gn/T/ipykernel_55387/1156190809.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  reduced_tech_pool = tech_pool.groupby("Category").apply(lambda x: x.sample(frac=0.5, random_state=42)).reset_index(drop=True)


In [13]:
jobs = pd.DataFrame()

for title in job_description["Job Title"].unique():
  jobs = pd.concat([jobs, job_description[job_description["Job Title"]==title].sample(1, random_state=42)])

jobs.reset_index(drop=True, inplace=True)

## saving dataset


In [14]:
job_description.to_csv(os.path.join(output_dir, "job_description.csv"), index=False)
talent_pool.to_csv(os.path.join(output_dir, "filtered_talent_pool.csv"), index=False)
jobs.to_csv(os.path.join(output_dir, "filtered_job_description.csv"), index=False)

# evaluate the resume


sanitary check


In [33]:
# jd = job_description[job_description["Job Title"]=="Machine Learning"]["Job Description"].values[0]
# jd_id = job_description[job_description["Job Title"]=="Machine Learning"]["Job ID"].values[0]
# cv = cleaned_resume.iloc[77]["Resume"]
# cv_id = cleaned_resume.iloc[77]["ID"]

In [36]:
# temperature = 0
# max_tokens = 2048

# groq_llm = ChatGroq(model="llama3-70b-8192", temperature=temperature, max_tokens=max_tokens)
# gpt_llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=temperature, max_tokens=max_tokens)
# anthropic_llm = ChatAnthropic(model="claude-3-5-sonnet-20240620", temperature=temperature, max_tokens=max_tokens)
# llama3_1_llm = ChatOllama(model="llama3.1:8b", temperature=temperature, max_tokens=max_tokens)

# resume_eval_prompt = PromptTemplate(
#     input_variables=["job_description", "resume"],
#     template=RESUME_EVALUATION_PROMPT
#     )

# # llama3
# groq_grader = resume_eval_prompt | groq_llm | JsonOutputParser()
# groq_result = groq_grader.invoke({"job_description": jd, "resume": cv})

# # gpt
# gpt_grader = resume_eval_prompt | gpt_llm | JsonOutputParser()
# gpt_result = gpt_grader.invoke({"job_description": jd, "resume": cv}) 

# # claude
# anthropic_grader = resume_eval_prompt | anthropic_llm | JsonOutputParser()
# anthropic_result = anthropic_grader.invoke({"job_description": jd, "resume": cv})

# llama3_grader = resume_eval_prompt | llama3_llm | JsonOutputParser()
# llama3_result = llama3_grader.invoke({"job_description": jd, "resume": cv})

# llama3_1_grader = resume_eval_prompt | llama3_1_llm | JsonOutputParser()
# llama3_1_result = llama3_1_grader.invoke({"job_description": jd, "resume": cv})

# model_results = {
    # "groq": groq_result,
    # "anthropic": anthropic_result,
    # "gpt": gpt_result,
    # "llama3_1": llama3_1_result
# }

# comparison_df = compare_model_outputs(model_results)

# # Display the pretty table
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
# print(comparison_df.to_string(index=False))

# save it to a CSV file:
# comparison_df.to_csv("model_comparison.csv", index=False)

In [17]:
# comparison_df

## set up the models


In [11]:
# groq_llm = ChatGroq(model="llama3-70b-8192", temperature=temperature, max_tokens=max_tokens)
gpt_llm = ChatOpenAI(model="chatgpt-4o-latest", temperature=temperature, max_tokens=max_tokens)
# anthropic_llm = ChatAnthropic(model="claude-3-5-sonnet-20240620", temperature=temperature, max_tokens=max_tokens)
# llama3_llm = ChatOllama(model="llama3", temperature=temperature, max_tokens=max_tokens)
 
resume_eval_prompt = PromptTemplate(
    input_variables=["job_description", "resume"],
    template=RESUME_EVALUATION_PROMPT
    )

gpt_grader = resume_eval_prompt | gpt_llm | JsonOutputParser()
# groq_grader = resume_eval_prompt | groq_llm | JsonOutputParser()
# anthropic_grader = resume_eval_prompt | anthropic_llm | JsonOutputParser()
# llama3_grader = resume_eval_prompt | llama3_llm | JsonOutputParser()

model_tuples = [
    ("gpt4o", gpt_grader),
    # ("anthropic", anthropic_grader),
    # ("llama3", llama3_grader)
    # ("groq", groq_grader)
]


In [12]:
jobs = pd.read_csv(os.path.join("output", "filtered_job_description.csv"))
talent_pool = pd.read_csv(os.path.join("output", "filtered_talent_pool.csv"))

In [13]:
def process_job_cv_pair(jod_data, cv_data):
    job_id, job_description = jod_data
    cv_id, cv = cv_data
    return evaluate_resume(model_tuples, job_description, cv, job_id, cv_id, output_dir)

def process_all_pairs():
    job_data = jobs[["Job ID", "Job Description"]].values
    cv_data = talent_pool[["ID", "Resume"]].values

    total_pairs = len(job_data) * len(cv_data)
    
    with ThreadPoolExecutor(max_workers=8) as executor:
        futures = [] 
        for job in job_data:
            for cv in cv_data:
                futures.append(executor.submit(process_job_cv_pair, job, cv))
            
        for future in tqdm(as_completed(futures), total=total_pairs, desc="Processing job-cv pairs"):
            try:
                future = future.result()
            except Exception as e:
                print(f"Error: {e}")


In [14]:
process_all_pairs()

Processing job-cv pairs:   0%|          | 0/705 [00:00<?, ?it/s]

Error with gpt4o for job_id: ca0539ba-835b-48fb-bed0-705d6668c372, cv_id: 508190c8-d2e1-4d77-b19d-11a1aaa07ebd. Error: Invalid json output: ```json
{
  "job_description_analysis": {
    "technical_skills": {
      "essential": ["ASP.NET", "SQL", "VBA", "VB.NET", "C#"],
      "advantageous": ["SSRS", "ERP systems", "WTS Paradigm"]
    },
    "soft_skills": ["communication", "teamwork", "independent judgment", "problem-solving"],
    "level_of_exp": "mid-level",
    "education": ["Bachelor’s Degree in related field"]
  },
  "resume_evaluation": {
    "original_scores": {
      "technical_skills": 20,
      "soft_skills": 50,
      "experience": 50,
      "education": 0
    },
    "missing_skills": ["ASP.NET", "SQL", "VB.NET", "C#", SSRS, "ERP systems", "WTS Paradigm"]
  },
  "deeper_analysis": {
    "inferred_experiences": {
      "SQL": "inferred from working on Excel Macro and MIS",
      "VBA": "inferred from working on Excel Macro"
    }
  },
  "recalibrated_scores": {
    "technical

## error log


In [18]:
model_name = [mod for mod, _ in model_tuples]

In [41]:

import re

log_file_dir = Path(output_dir, "evaluation_log.txt")

with open(log_file_dir, "r") as file:
    log_content = file.read()

error_entries = re.findall(f'ERROR - Error with .* for job_id:.*, cv_id:.*(?=\.)', log_content)
uuid_pattern =  r"^[0-9a-fA-F]{8}-[0-9a-fA-F]{4}-[1-5][0-9a-fA-F]{3}-[89abAB][0-9a-fA-F]{3}-[0-9a-fA-F]{12}$"


In [42]:
entry = error_entries[0]
print(entry)

job_id = re.search(f'job_id: ({uuid_pattern})', entry)
cv_id = re.search(f'cv_id: ({uuid_pattern})', entry)

job_id, cv_id

ERROR - Error with gpt4o for job_id: ca0539ba-835b-48fb-bed0-705d6668c372, cv_id: 508190c8-d2e1-4d77-b19d-11a1aaa07ebd


(None, None)

In [46]:
error_pattern = r'^\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2} - ERROR -'
info_pattern =r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2} - INFO -' 
sections = re.split(error_pattern, log_content, flags=re.MULTILINE)
sections = [item.strip() for item in sections if item.strip().startswith("Error with ")]
sections = [re.split(info_pattern, sec)[0] for sec in sections]
json_strs = [re.search(r'(\{.*\})', sec, re.DOTALL).group(1) for sec in sections]

In [47]:
json_strs

['{\n  "job_description_analysis": {\n    "technical_skills": {\n      "essential": ["ASP.NET", "SQL", "VBA", "VB.NET", "C#"],\n      "advantageous": ["SSRS", "ERP systems", "WTS Paradigm"]\n    },\n    "soft_skills": ["communication", "teamwork", "independent judgment", "problem-solving"],\n    "level_of_exp": "mid-level",\n    "education": ["Bachelor’s Degree in related field"]\n  },\n  "resume_evaluation": {\n    "original_scores": {\n      "technical_skills": 20,\n      "soft_skills": 50,\n      "experience": 50,\n      "education": 0\n    },\n    "missing_skills": ["ASP.NET", "SQL", "VB.NET", "C#", SSRS, "ERP systems", "WTS Paradigm"]\n  },\n  "deeper_analysis": {\n    "inferred_experiences": {\n      "SQL": "inferred from working on Excel Macro and MIS",\n      "VBA": "inferred from working on Excel Macro"\n    }\n  },\n  "recalibrated_scores": {\n    "technical_skills": 40,\n    "soft_skills": 50,\n    "experience": 50,\n    "education": 0\n  },\n  "assessment": {\n    "suitabil

In [49]:
import json

for json_str in json_strs:
  try:
    result = json.loads(json_str)
    print(result) 
  except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")
    print(json_str)

Error decoding JSON: Expecting value: line 18 column 58 (char 592)
{
  "job_description_analysis": {
    "technical_skills": {
      "essential": ["ASP.NET", "SQL", "VBA", "VB.NET", "C#"],
      "advantageous": ["SSRS", "ERP systems", "WTS Paradigm"]
    },
    "soft_skills": ["communication", "teamwork", "independent judgment", "problem-solving"],
    "level_of_exp": "mid-level",
    "education": ["Bachelor’s Degree in related field"]
  },
  "resume_evaluation": {
    "original_scores": {
      "technical_skills": 20,
      "soft_skills": 50,
      "experience": 50,
      "education": 0
    },
    "missing_skills": ["ASP.NET", "SQL", "VB.NET", "C#", SSRS, "ERP systems", "WTS Paradigm"]
  },
  "deeper_analysis": {
    "inferred_experiences": {
      "SQL": "inferred from working on Excel Macro and MIS",
      "VBA": "inferred from working on Excel Macro"
    }
  },
  "recalibrated_scores": {
    "technical_skills": 40,
    "soft_skills": 50,
    "experience": 50,
    "education": 0
  }

In [51]:
json_str[592:600]

'SSRS, "E'

In [54]:
import json
import subprocess
import re

def fix_json(json_string):
    # Try to parse the JSON first
    try:
        json.loads(json_string)
        return json_string, None  # If it's valid, return as is
    except json.JSONDecodeError as e:
        error_message = str(e)
        print(f"Original error: {error_message}")

    # Use jsonlint to get more detailed error information
    result = subprocess.run(['jsonlint', '-c'], input=json_string, text=True, capture_output=True)
    
    if result.returncode != 0:
        error_output = result.stderr
        
        # Extract line and column information
        match = re.search(r'line (\d+), col (\d+)', error_output)
        if match:
            line, col = map(int, match.groups())
            
            # Split the JSON string into lines
            lines = json_string.split('\n')
            
            # Fix common errors
            if 'Expecting' in error_output:
                if 'value' in error_output:
                    # Missing value after colon
                    lines[line-1] = lines[line-1][:col-1] + ' null' + lines[line-1][col-1:]
                elif 'property name' in error_output:
                    # Missing property name or extra comma
                    if lines[line-1].strip().endswith(','):
                        lines[line-1] = lines[line-1].rstrip()[:-1]  # Remove trailing comma
                    else:
                        lines[line-1] = lines[line-1][:col-1] + '"missing_property": null' + lines[line-1][col-1:]
            elif 'Unexpected' in error_output:
                if '}' in error_output or ']' in error_output:
                    # Extra comma before closing bracket
                    lines[line-1] = lines[line-1][:col-1] + lines[line-1][col:]
            
            # Join the lines back together
            fixed_json = '\n'.join(lines)
            
            # Recursively try to fix any remaining errors
            return fix_json(fixed_json)
    
    return json_string, "Failed to fix JSON errors"

# Example usage
json_with_errors = '''{
  "job_description_analysis": {
    "technical_skills": {
      "essential": ["ASP.NET", "SQL", "VBA", "VB.NET", "C#"],
      "advantageous": ["SSRS", "ERP systems", "WTS Paradigm"]
    },
    "soft_skills": ["communication", "teamwork", "independent judgment", "problem-solving"],
    "level_of_exp": "mid-level",
    "education": ["Bachelor's Degree in related field"]
  },
  "resume_evaluation": {
    "original_scores": {
      "technical_skills": 20,
      "soft_skills": 50,
      "experience": 50,
      "education": 0
    },
    "missing_skills": ["ASP.NET", "SQL", "VB.NET", "C#", SSRS, "ERP systems", "WTS Paradigm"]
  },
  "deeper_analysis": {
    "inferred_experiences": {
      "SQL": "inferred from working on Excel Macro and MIS",
      "VBA": "inferred from working on Excel Macro"
    }
  },
  "recalibrated_scores": {
    "technical_skills": 40,
    "soft_skills": 50,
    "experience": 50,
    "education": 0
  },
  "assessment": {
    "suitability": "no",
    "strengths": ["Experience with Excel Macros", "Experience in MIS reporting"],
    "potential_concerns": ["Lack of direct experience with ASP.NET, VB.NET, C#", "No Bachelor's Degree in a related field"],
    "missing_skills": ["ASP.NET", "VB.NET", "C#", "SSRS", "ERP systems", "WTS Paradigm"],
    "reasons": [
      "The candidate lacks essential technical skills such as ASP.NET, VB.NET, and C#.",
      "The candidate does not have a Bachelor's Degree in a related field, which is a requirement.",
      "The candidate's experience is more aligned with business analysis and MIS reporting rather than software development."
    ]
  },
  "feedback": {
    "significant_mismatches": [
      "The candidate should consider gaining experience in ASP.NET, VB.NET, and C# to align better with the job requirements.",
      "Pursuing a Bachelor's Degree in a related field or obtaining relevant certifications could improve the candidate's qualifications."
    ]
  },
  "summary": {
    "evaluation": "The candidate is not a suitable match for the Programmer/Software Developer role due to significant gaps in essential technical skills and educational qualifications. The candidate's experience is more aligned with business analysis rather than software development, which is the primary focus of the job description.",
    "decision": "no"
  }
}'''

fixed_json, error = fix_json(json_with_errors)
if error:
    print(f"Error: {error}")
else:
    print("Fixed JSON:")
    print(fixed_json)

# Verify the fixed JSON
try:
    json.loads(fixed_json)
    print("\nJSON is now valid!")
except json.JSONDecodeError as e:
    print(f"\nJSON is still invalid. Error: {e}")

Original error: Expecting value: line 18 column 58 (char 592)


FileNotFoundError: [Errno 2] No such file or directory: 'jsonlint'

In [53]:
!pip install jsonlint

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.7/103.7 kB 3.2 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
